Import Libraries

In [72]:
import numpy as np
import pandas as pd
import spacy
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.utils import np_utils

In [73]:
print(tf.__version__)

2.2.0


Loading Features

In [74]:
from numpy import genfromtxt
features = genfromtxt('features.csv', delimiter=',')
encoded_labels = genfromtxt('classes.csv', delimiter=',')

In [75]:
features

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 1.800e+01, 4.547e+03,
        4.548e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 8.200e+01, 5.000e+00,
        8.000e+01],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 5.300e+01, 3.590e+02,
        6.800e+02],
       ...,
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 4.630e+02, 5.970e+02,
        9.481e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e+00, 1.310e+02,
        5.760e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.180e+02, 3.660e+02,
        1.000e+00]])

In [76]:
encoded_labels.astype('int64')

array([[0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0]])

Splitting Our DATA

In [77]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), "Train y: \t\t{}".format(train_y.shape),
      "\nValidation set: \t{}".format(val_x.shape),"valid y: \t\t{}".format(val_y.shape),
      "\nTest set: \t\t{}".format(test_x.shape),"Test y: \t\t{}".format(test_y.shape)) 

			Feature Shapes:
Train set: 		(11656, 15) Train y: 		(11656, 3) 
Validation set: 	(1457, 15) valid y: 		(1457, 3) 
Test set: 		(1457, 15) Test y: 		(1457, 3)


Importing Layers from Keras

In [80]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import SpatialDropout1D
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [81]:
no_of_words = 9481

**MODEL 1:**


In [82]:
model1 = Sequential()
model1.add(Embedding(no_of_words, 100,input_length=15))
#model1.add(SpatialDropout1D(0.2))
model1.add(LSTM(128))
model1.add(Dropout(0.5))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.5))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(3, activation='softmax'))
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
filepath = "model.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

model1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 15, 100)           948100    
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)               0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 128)               512       
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_23 (Dropout)         (None, 256)              

CHECKPOINT

In [85]:

checkpoint = ModelCheckpoint('model.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

**MODEL 1: TRAIN**

In [86]:
batch_size=40
print('Train...')
model1.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=15,
          validation_data=(val_x, val_y),verbose=1,callbacks=[checkpoint])


Train...
Train on 11656 samples, validate on 1457 samples
Epoch 1/15
11656/11656 [==============================] - 10s 856us/step - loss: 0.6704 - accuracy: 0.7195 - val_loss: 0.6685 - val_accuracy: 0.6966

Epoch 00001: val_loss improved from inf to 0.66845, saving model to model.h5
Epoch 2/15
11656/11656 [==============================] - 10s 844us/step - loss: 0.4965 - accuracy: 0.8074 - val_loss: 0.6155 - val_accuracy: 0.7612

Epoch 00002: val_loss improved from 0.66845 to 0.61551, saving model to model.h5
Epoch 3/15
11656/11656 [==============================] - 10s 845us/step - loss: 0.4250 - accuracy: 0.8354 - val_loss: 0.6271 - val_accuracy: 0.7653

Epoch 00003: val_loss did not improve from 0.61551
Epoch 4/15
11656/11656 [==============================] - 10s 851us/step - loss: 0.3649 - accuracy: 0.8602 - val_loss: 0.6506 - val_accuracy: 0.7550

Epoch 00004: val_loss did not improve from 0.61551
Epoch 5/15
11656/11656 [==============================] - 10s 840us/step - loss: 0

**MODEL 1 : EVALUATE**

In [87]:

score, acc = model1.evaluate(test_x, test_y,
                            batch_size=batch_size)
print(score,acc)

1457/1457 [==============================] - 0s 123us/step
1.0359639361776878 0.7261496186256409
